# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

The following will check that your code outputs the expected min cost flow values on several test instances.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """

    edges = dict()
    nodes = dict()
    with open(infile, 'r') as graph:
        for line in graph.readlines():
            line = line.split(' ')
            if line[0] == 'n':
                value = dict()
                value['demand'] = int(line[2])
                nodes[int(line[1])] = value['demand']
                
            if line[0] == 'a':
                v1 = int(line[1])
                v2 = int(line[2])
                e = (v1, v2)
                if e not in edges.keys():
                    constraint = dict()
                    constraint['capacity'] = [int(line[4])]
                    constraint['weight'] = [float(line[5])]
                    edges[e] = constraint
                else:
                    edges[e]['capacity'].append(int(line[4]))
                    edges[e]['weight'].append(float(line[5]))
                    
            if line[0] == 'p':
                idnumber = int(line[2])
                edgenumber = int(line[3])
    
    

    for e in edges.keys(): ##add new edges
        if len(edges[e]['capacity']) >= 2:
            for k in range(len(edges[e]['capacity'])):
                idnumber = idnumber + 1
                v1 = e[0]
                v2 = e[1]
                first = (v1,idnumber)  ##two new edges
                second = (idnumber, v2) 
                constraint1 = dict()
                constraint2 = dict()                
                constraint1['capacity'] = edges[e]['capacity'][k]
                constraint2['capacity'] = edges[e]['capacity'][k]
                constraint1['weight'] = edges[e]['weight'][k]/2.
                constraint2['weight'] = edges[e]['weight'][k]/2.               
                edges[first] = constraint1
                edges[second] = constraint2
            del edges[e]

    G = nx.DiGraph() ##construct a new graph
    
    for v in nodes: ##add nodes
        G.add_node(v)
        G.node[v]['demand'] = nodes[v]
        
    for e in edges.keys(): ##add edges
        v1=e[0]
        v2=e[1]
        G.add_edge(v1, v2)
        G.edge[v1][v2]['capacity'] = edges[(v1, v2)]['capacity']
        G.edge[v1][v2]['weight'] = edges[(v1, v2)]['weight']       
 
    return G

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    model = pulp.LpProblem("min-cost problem", pulp.LpMinimize)
    weight = dict()

    ##set the variables
    flow_dic=[str(edge) for edge in G.edges()]

    flow = pulp.LpVariable.dict('flow', flow_dic, lowBound = 0, cat = pulp.LpInteger)
    ##set the weight  

    for e in G.edges():
        weight[str(e)] = G.get_edge_data(e[0], e[1])['weight']

    ##set the constraints
    for edge in G.edges():
        model += flow[str(edge)] <= G.get_edge_data(edge[0], edge[1])['capacity']
    for node in G.nodes():
        if G.node[node] == {}:
            demand = 0
        else:
            demand = G.node[node]['demand']
        
        flow_in = 0
        flow_out = []
        for edge in G.edges():
            if node == edge[1]:
                flow_in += pulp.lpSum(flow[str(edge)])
            if node == edge[0]:
                flow_out += pulp.lpSum(flow[str(edge)])
        model += flow_in - flow_out == demand

    ##set the objective
    cost=0
    for edge in G.edges():
        cost+=weight[str(edge)]*flow[str(edge)]
    model += cost

    model.solve()
    pulp.LpStatus[model.status]

    return(pulp.value(model.objective))
    pass

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
